<a href="https://colab.research.google.com/github/Axl11475581/FreeCodeCamp-Tensor-Flow-Repo/blob/main/Cat_%26_Dog_Classifier_test_ground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Get project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

!unzip cats_and_dogs.zip

PATH = 'cats_and_dogs'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Get number of files in each directory. The train and validation directories
# each have the subdirecories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [40]:
train_image_generator = ImageDataGenerator(rescale=1/225)
validation_image_generator = ImageDataGenerator(rescale=1/225)
test_image_generator = ImageDataGenerator(1/225)

train_data_gen = train_image_generator.flow_from_directory(
directory=train_dir,
target_size=(IMG_HEIGHT, IMG_WIDTH),
color_mode= 'rgb',
batch_size=batch_size,
class_mode= 'binary',
shuffle=True,
seed=None
)
val_data_gen =validation_image_generator.flow_from_directory(
directory=validation_dir,
target_size=(IMG_HEIGHT, IMG_WIDTH),
color_mode= 'rgb',
batch_size=batch_size,
class_mode= 'binary',
shuffle=True,
seed=None
)
test_data_gen = test_image_generator.flow_from_directory(
directory=test_dir,
target_size=(IMG_HEIGHT, IMG_WIDTH),
color_mode= 'rgb',
batch_size=batch_size,
class_mode= 'input',
shuffle=False,
seed=None
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
